# Crossword Puzzle Generator 
## Summary:
This project tries to fill out any crossword puzzle with a valid input in a managible amount of time. I have scarped New York Times Crosswords for the past 20+ years however i can note share that data because of copyright issues i might run into, so in this notebook i will just be using a dictionary of English words.

In [1]:
import nltk
from nltk.corpus import words
# look for "words" under the "Corpora" tab
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
list_of_words = words.words()

In [3]:
len(list_of_words)

236736

In [4]:
list_of_words[:5]

['A', 'a', 'aa', 'aal', 'aalii']

In [5]:
import numpy as np

In [6]:
# these fumctions put the zero padding around the puzzle
def pad_with(vector, pad_width, iaxis, kwargs):
    pad_value = kwargs.get('padder', 10)
    vector[:pad_width[0]] = pad_value
    vector[-pad_width[1]:] = pad_value


def pad_puzzle(puzzle):
    """
    adds a layer of zeros around the puzzle
    """
    return np.pad(puzzle, 1, pad_with, padder=0)

In [7]:
simple_ex = np.ones((5,5),dtype=str)

In [8]:
simple_ex

array([['1', '1', '1', '1', '1'],
       ['1', '1', '1', '1', '1'],
       ['1', '1', '1', '1', '1'],
       ['1', '1', '1', '1', '1'],
       ['1', '1', '1', '1', '1']], dtype='<U1')

In [9]:
pad_puzzle(simple_ex)

array([['0', '0', '0', '0', '0', '0', '0'],
       ['0', '1', '1', '1', '1', '1', '0'],
       ['0', '1', '1', '1', '1', '1', '0'],
       ['0', '1', '1', '1', '1', '1', '0'],
       ['0', '1', '1', '1', '1', '1', '0'],
       ['0', '1', '1', '1', '1', '1', '0'],
       ['0', '0', '0', '0', '0', '0', '0']], dtype='<U1')

In [10]:
simple_ex_padded = pad_puzzle(simple_ex)

In [11]:
# horizontal word length 
def get_h_word_length(puzzle, v, h):
    """
    find the length of a word, by seeing how many ones are in front,
    and in back

    :param h: horizontal_pos
    :param v: vertical_pos
    :return: horizontal word length
    """
    word_length = 0

    for i in range(17):
        if puzzle[v][h + i] != '0':
            word_length += 1
        else:
            break

    for i in range(17):
        if puzzle[v][h - i] != '0':
            word_length += 1
        else:
            break

    return word_length - 1


In [12]:
# vertical word length 

def get_v_word_length(puzzle, v, h):
    """
    find the length of a word, by seeing how many ones are in front,
    and in back

    :param h: horizontal_pos
    :param v: vertical_pos
    :return: horizontal word length
    """
    word_length = 0

    for i in range(17):
        if puzzle[v + i][h] != '0':
            word_length += 1
        else:
            break

    for i in range(17):
        if puzzle[v - i][h] != '0':
            word_length += 1
        else:
            break

    return word_length - 1

In [16]:
get_h_word_length(simple_ex_padded,1,1)

5

In [17]:
def get_h_frag(puzzle, v, h, h_word_len):
    '''
    gets all letter behind the letter in possion v,h
    
    '''
    word = []
    for i in range(h_word_len):
        if puzzle[v][h - i] == '0':
            break
        else:
            word.append(puzzle[v][h - i])

    word_frag = ''

    for m in word:
        if m != '0' and m != '1':
            word_frag += m
    h_word_frag = word_frag[::-1]
    return h_word_frag


def get_v_frag(puzzle, v, h, v_word_len):
    '''
    gets all letter abouve the letter in possion v,h
    
    '''
    word = []
    for i in range(v_word_len):
        if puzzle[v - i][h] == '0':
            break
        else:
            word.append(puzzle[v - i][h])
    word_frag = ''
    for m in word:
        if m != '0' and m != '1':
            word_frag += m

    v_word_frag = word_frag[::-1]
    return v_word_frag

In [18]:
def get_possible_h_letters(h_word_frag, h_word_length, word_list):
    '''
    gets all letters that start with the word frag
    and have a specified length
    
    '''
    possible_letter_h = []
    for word in word_list:
        if word.startswith(str(h_word_frag)) and len(word) == h_word_length:
            possible_letter_h.append(word[len(h_word_frag)])

    return possible_letter_h


def get_possible_v_letters(v_word_frag, v_word_length, word_list):
    '''
    gets all letters that start with the word frag
    and have a specified length
    
    '''
    possible_letter_v = []
    for word in word_list:
        if word.startswith(str(v_word_frag)) and len(word) == v_word_length:
            possible_letter_v.append(word[len(v_word_frag)])

    return possible_letter_v

In [19]:
def prob_of_letter(possible_letter_h, possible_letter_v, puzzle):
    '''
    gets the joint probability of possible vertical letters as well 
    as possible horizontal letters
    
    '''
    possible_letters = (pd.Series(possible_letter_h).value_counts() / len(possible_letter_h) * pd.Series(
        possible_letter_v).value_counts() / len(possible_letter_v)).dropna()

    if len(np.argwhere(puzzle == '1')) < 130:
        if np.random.uniform(0, 1) < .2:
            return possible_letters
        else:
            return possible_letters.sort_values(ascending=False)[:2]
    else:
        return possible_letters


In [20]:
def create(puzzle, word_list, h_past, v_past):
    '''
    creats a valid crossword with valid words going across and down
    
    '''
    print(puzzle, '\n')
    try:
        value = np.argwhere(puzzle == '1')[0]
    except IndexError:
        return True

    print(len(np.argwhere(puzzle == '1')))
    if len(np.argwhere(puzzle == '1')) < 1:
        return True
    else:
        row = value[0]
        column = value[1]

        h_word_length = get_h_word_length(puzzle, row, column)
        v_word_length = get_v_word_length(puzzle, row, column)

        h_frag = get_h_frag(puzzle, row, column, h_word_length)
        v_frag = get_v_frag(puzzle, row, column, v_word_length)
        
        # caches some of the possible letters if it has seen it befor 
        if h_frag + str(h_word_length) in list(h_past.keys()):
            pos_h = h_past[h_frag + str(h_word_length)]
        else:
            pos_h = get_possible_h_letters(h_frag, h_word_length, word_list)
            h_past[h_frag + str(h_word_length)] = pos_h

        if v_frag + str(v_word_length) in list(v_past.keys()):
            pos_v = v_past[v_frag + str(v_word_length)]
        else:
            pos_v = get_possible_v_letters(v_frag, v_word_length, word_list)
            v_past[v_frag + str(v_word_length)] = pos_v

        possible_letters = prob_of_letter(pos_h, pos_v, puzzle)

        try:
            p_letters = np.random.choice(possible_letters.index, len(possible_letters.index), replace=False,
                                         p=possible_letters / sum(possible_letters))
        except ValueError:
            p_letters = []

        p_letters = p_letters[:2]
        print(p_letters)
        for letter in p_letters:
            print(letter)
            puzzle[row][column] = letter

            create(puzzle, word_list,h_past,v_past)

            if len(np.argwhere(puzzle == '1')) < 1:
                return puzzle

            print('\n back \n')
            puzzle[row][column] = '1'


        return False

In [25]:
import pandas as pd 
simple_ex = np.ones((5,5),dtype=str)
simple_ex_padded = pad_puzzle(simple_ex)

h_past = {}
v_past = {}

data = create(simple_ex_padded, list_of_words,h_past,v_past)

[['0' '0' '0' '0' '0' '0' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '0' '0' '0' '0' '0' '0']] 

25
['s' 'c']
s
[['0' '0' '0' '0' '0' '0' '0']
 ['0' 's' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '0' '0' '0' '0' '0' '0']] 

24
['t' 'a']
t
[['0' '0' '0' '0' '0' '0' '0']
 ['0' 's' 't' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '0' '0' '0' '0' '0' '0']] 

23
['a' 'r']
a
[['0' '0' '0' '0' '0' '0' '0']
 ['0' 's' 't' 'a' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '1' '1' '1' '1' '1' '0']
 ['0' '0' '0' '0' '0' '0' '0']] 

22
['r' 'g']
r
[['0' '0' '0' '0' '0' '0' '0']
 ['0' 's' 't' 'a' 'r' '1' '0'

In [26]:
simple_ex_padded

array([['0', '0', '0', '0', '0', '0', '0'],
       ['0', 's', 't', 'a', 'r', 't', '0'],
       ['0', 't', 'a', 'l', 'e', 'r', '0'],
       ['0', 'a', 'l', 'i', 's', 'o', '0'],
       ['0', 'r', 'e', 's', 'e', 't', '0'],
       ['0', 't', 'r', 'o', 't', 'h', '0'],
       ['0', '0', '0', '0', '0', '0', '0']], dtype='<U1')

technically the complexity of this algorithm is O(2 ^ number of 1's). so in the 5 by 5 case it is O(2^25). so that sucks so i added a number of heuristics as well as caching previous results and that significantly speed up the algorithm. this algorithm could solve a NYT crossword style crossword (15 by 15) in a reasonable amount of time (anywhere from 30 min to 8 hours, based on my tests) the complexity of a 15 by 15 puzzle is O(2^~180) (usually 180 letters in a NYT crossword)